## Testing with the OpenAI API

### Simple Usage

In [ ]:
from openai import OpenAI
from config import API_KEY
client = OpenAI(api_key=API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="In the realm of code where patterns entwine,\nThere lies a concept both elegant and fine.\nRecursion, a dance of functions divine,\nA mystical loop that intertwines.\n\nA function that calls itself, oh so clever,\nAn endless cycle that seems to go on forever.\nWith each iteration, it solves a piece,\nUnraveling problems, granting release.\n\nLike a fractal, repeating patterns unfold,\nA beauty in simplicity, a story told.\nDivide and conquer, the recursive quest,\nBreaking problems down into the smallest jest.\n\nBase cases as anchors, to stop the recursion flow,\nWithout them, chaos would surely grow.\nSo mind the stack, the memory's pace,\nA balance to keep in this recursive race.\n\nSo behold the magic of recursion's embrace,\nA concept in coding, full of grace.\nIt bends the rules, defies convention,\nA loop like no other, a coder's invention.", role='assistant', function_call=None, tool_ca

In [ ]:
def get_user_prompt(role = 'user', user_prompt = ""):
  """Gets user prompt and appends location request."""
  # Append "including coordinates" to ensure location retrieval
  user_prompt += "include a dictionary at the end of your response with the following format: {place: coordinates}. Everytime you mention a place in your response, add it to the dictionary"
  message_dict = {"role": role, "content": user_prompt}
  return message_dict
    
# completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="tell me about the eiffel tower")])

# print(completion.choices[0].message.content)

In [ ]:
import re

# Function to extract coordinates from text using regular expressions
# Function to extract coordinates from text using regular expressions
def extract_coordinates(text):
    # Regular expression pattern to match coordinates in the format "48.8584° N, 2.2945° E"
    pattern = r"(-?\d+\.\d+)° (N|S), (-?\d+\.\d+)° (E|W)"
    match = re.search(pattern, text)
    if match:
        latitude = float(match.group(1)) * (-1 if match.group(2) == 'S' else 1)
        longitude = float(match.group(3)) * (-1 if match.group(4) == 'W' else 1)
        return latitude, longitude
    else:
        return (-1, -1)

In [ ]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="tell me about the eiffel tower")])
print(f"Latitude: {extract_coordinates(completion.choices[0].message.content)[0]}\nLongitude: {extract_coordinates(completion.choices[0].message.content)[1]}")

Latitude: 48.8584
Longitude: 2.2945


![Eiffel Tower Maps](eiffeltowermaps.png)

### Multiple Locations

In [ ]:
from openai import OpenAI
from config import API_KEY, ASSISTANT_ID
client = OpenAI(api_key=API_KEY)

In [ ]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="what museums should I visit in Paris?")])
print(completion.choices[0].message.content)

Paris is home to some of the most famous and impressive museums in the world. Here are a few that you should definitely consider visiting:

1. The Louvre - This iconic museum is home to thousands of works of art, including the famous Mona Lisa. It is located at coordinates: {Louvre: 48.8606° N, 2.3376° E}.

2. Musée d'Orsay - Housed in a stunning former railway station, this museum is home to an impressive collection of Impressionist and Post-Impressionist art. It is located at coordinates: {Musée d'Orsay: 48.8599° N, 2.3266° E}.

3. Centre Pompidou - This modern art museum is known for its unique architecture and impressive collection of contemporary art. It is located at coordinates: {Centre Pompidou: 48.8606° N, 2.3521° E}.

4. Musée Rodin - This museum is dedicated to the works of the famous sculptor Auguste Rodin and is located in his former home and studio. It is located at coordinates: {Musée Rodin: 48.8554° N, 2.3155° E}.

5. Musée de l'Orangerie - This museum is known for its 

### Testing with Asistants

In [ ]:
#already created an assistant, checking list of assistants
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_ERzvBehgZHH410kiTOuGtlNJ', created_at=1710757402, description=None, file_ids=[], instructions='you are a travel guide, an expert in geography and tourism, well suited to inform travellers on geographical, historical and cultural data', metadata={}, model='gpt-3.5-turbo-0125', name='AliveBot', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])]


In [ ]:
my_assistant = client.beta.assistants.retrieve(assistant_id=ASSISTANT_ID)

In [ ]:
'''in case we want to modify the params'''

# my_updated_assistant = client.beta.assistants.update(
#   "asst_abc123",
#   instructions="You are an HR bot, and you have access to files to answer employee questions about company policies. Always response with info from either of the files.",
#   name="HR Helper",
#   tools=[{"type": "retrieval"}],
#   model="gpt-4",
#   file_ids=["file-abc123", "file-abc456"],
# )

### Experimenting with threads

In [ ]:
#create an empty thread

empty_thread = client.beta.threads.create()
print(empty_thread)

Thread(id='thread_ELhgxetceIT0D8ljO4UtYe2Q', created_at=1710759917, metadata={}, object='thread')


In [ ]:
#create a user message within the thread

thread_message = client.beta.threads.messages.create(
  "thread_ELhgxetceIT0D8ljO4UtYe2Q",
  role="user",
  content="How does AI work? Explain it in simple terms.",
)
print(thread_message)

ThreadMessage(id='msg_yZSofQlYKHbgFAWf8FhrUhez', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='How does AI work? Explain it in simple terms.'), type='text')], created_at=1710759944, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_ELhgxetceIT0D8ljO4UtYe2Q')


In [ ]:
#running the thread with the agent

run = client.beta.threads.runs.create(
  thread_id="thread_ELhgxetceIT0D8ljO4UtYe2Q",
  assistant_id=ASSISTANT_ID
)

In [ ]:
#retrieving the thread

my_thread = client.beta.threads.messages.list("thread_ELhgxetceIT0D8ljO4UtYe2Q")
print(my_thread.data)

[ThreadMessage(id='msg_gUV70YdFQuXEWE37tY56kGFH', assistant_id='asst_ERzvBehgZHH410kiTOuGtlNJ', content=[MessageContentText(text=Text(annotations=[], value="Sure! I'll explain how AI works in simple terms.\n\nArtificial Intelligence, or AI, is a field of computer science that aims to create machines that can perform tasks that typically require human intelligence. AI systems are designed to learn from data, adapt to new inputs, and perform tasks without being explicitly programmed to do so.\n\nAt the core of AI are algorithms, which are sets of rules and instructions that guide the AI system in solving problems or making decisions. These algorithms use data to learn and improve over time, a process known as machine learning.\n\nMachine learning algorithms can be categorized into two main types: supervised learning and unsupervised learning. In supervised learning, the algorithm is trained on labeled data, meaning the correct answers are provided during training. In unsupervised learnin